
# Strands Agent Graph를 사용한 멀티 에이전트 시스템 구축
멀티 에이전트 시스템은 여러 전문화된 AI 에이전트가 함께 작업하여 조정된 협업을 통해 복잡한 문제를 해결하는 시스템입니다. 각 에이전트는 특정 능력과 역할을 가지며, 명시적인 통신 경로를 통해 연결됩니다.

이 실습에서는 Strands Agent SDK를 사용하여 멀티 에이전트 시스템을 구축하는 방법을 배웁니다. 기본 개념부터 고급 구현까지 진행하며, 다양한 토폴로지와 실제 애플리케이션을 탐색합니다.

**학습 목표:**
이 노트북을 완료하면 다음을 할 수 있게 됩니다:
- 에이전트 그래프의 세 가지 핵심 구성 요소(노드, 엣지, 조건) 이해
- 특정 에이전트 간에 타겟팅된 메시지 전송
- 멀티 에이전트 네트워크 모니터링 및 제어
- 실제 시나리오를 위한 전문화된 에이전트 시스템 설계

## 전제 조건

- Python 3.10+
- Amazon Bedrock에서 Anthropic Claude 3.7이 활성화된 AWS 계정
- Amazon Bedrock 사용 권한이 있는 IAM 역할
- AI 에이전트 및 프롬프트 엔지니어링에 대한 기본 이해

## 설정 및 설치

시작하기 전에 `strands-agents`와 `strands-agents-tools`에 필요한 패키지를 설치해보겠습니다

In [ ]:
%pip install -r requirements.txt

### 필요한 패키지 가져오기

다음으로 필요한 패키지를 가져올 수 있습니다

In [ ]:
from strands import Agent

## 에이전트 그래프 구성 요소 이해하기
에이전트 그래프는 조정된 협업을 통해 복잡한 문제를 해결하도록 설계된 상호 연결된 AI 에이전트의 구조화된 네트워크입니다. 각 에이전트는 특정 능력을 가진 전문화된 노드를 나타내며, 에이전트 간의 연결은 명시적인 통신 경로를 정의합니다.

구축을 시작하기 전에 에이전트 그래프의 세 가지 주요 구성 요소를 이해해보겠습니다:

### 1. 노드(Nodes) - 에이전트
각 노드는 다음을 가진 AI 에이전트를 나타냅니다:
- **식별자(Identity)**: 그래프 내에서의 고유 식별자
- **역할(Role)**: 전문화된 기능 또는 목적
- **시스템 프롬프트(System Prompt)**: 에이전트의 동작을 정의하는 지침
- **도구(Tools)**: 에이전트가 사용할 수 있는 능력

### 2. 엣지(Edges) - 연결
엣지는 다음을 가진 통신 경로를 정의합니다:
- **방향(Direction)**: 단방향 또는 양방향 정보 흐름
- **조건(Condition)**: 엣지를 통과해야 하는지 결정하는 선택적 함수
- **종속성(Dependencies)**: 노드 간의 실행 순서와 데이터 흐름 정의

### 3. GraphBuilder
GraphBuilder는 그래프 구성을 위한 간단한 인터페이스를 제공합니다:
- **add_node()**: 에이전트 또는 멀티 에이전트 시스템을 노드로 추가
- **add_edge()**: 노드 간의 종속성 생성
- **set_entry_point()**: 실행을 위한 시작 노드 정의
- **build()**: Graph 인스턴스 검증 및 생성



### 기본 처리

정의된 역할에 따라 출력이 달라지는 두 개의 서로 다른 에이전트가 하나의 작업을 처리하는 간단한 예제부터 시작해보겠습니다. 노드의 실행 순서와 Strands SDK를 사용하면 필요한 경우 단일 노드에서만 명시적으로 응답을 받을 수 있다는 사실을 살펴보세요. 아키텍처는 다음과 같습니다:

<div style="text-align:left">
    <img src="images/basic.png" width="55%" />
</div>

In [ ]:
# agent_graph 기능을 가진 에이전트 초기화
from strands.multiagent import GraphBuilder

# 전문화된 에이전트 생성
coordinator = Agent(name="coordinator", system_prompt="당신은 전문가들을 조정하는 연구팀 리더입니다. 간단한 분석을 제공하고, 후속 조치는 필요하지 않습니다")
analyst = Agent(name="data_analyst", system_prompt="당신은 통계 분석을 전문으로 하는 데이터 분석가입니다. 간단한 분석을 제공하고, 후속 조치는 필요하지 않습니다")
domain_expert = Agent(name="domain_expert", system_prompt="당신은 깊은 주제 지식을 가진 도메인 전문가입니다. 간단한 분석을 제공하고, 후속 조치는 필요하지 않습니다")

# 그래프 구축
builder = GraphBuilder()

# 노드 추가
builder.add_node(coordinator, "team_lead")
builder.add_node(analyst, "analyst")
builder.add_node(domain_expert, "expert")

# 엣지(종속성) 추가
builder.add_edge("team_lead", "analyst")
builder.add_edge("team_lead", "expert")

# 진입점 설정 (선택사항 - 지정하지 않으면 자동 감지됨)
builder.set_entry_point("team_lead")

# 그래프 구축
graph = builder.build()

# 새로 구축된 그래프에서 작업 실행
result = graph("원격 근무가 직원 생산성에 미치는 영향을 분석하세요. 간단한 분석을 제공하고, 후속 조치는 필요하지 않습니다")
print("\n")
print("============================================================")
print("============================================================")

print(f"응답: {result}")

print("=============노드 실행 순서:=================================")
print("============================================================")

# 어떤 노드가 어떤 순서로 실행되었는지 확인
for node in result.execution_order:
    print(f"실행됨: {node.node_id}")

print("=============그래프 메트릭:=================================")
print("============================================================")


# 성능 메트릭 가져오기
print(f"총 노드 수: {result.total_nodes}")
print(f"완료된 노드 수: {result.completed_nodes}")
print(f"실패한 노드 수: {result.failed_nodes}")
print(f"실행 시간: {result.execution_time}ms")
print(f"토큰 사용량: {result.accumulated_usage}")


# 특정 노드의 결과 가져오기
print("\n")
print("=============전문가 노드 결과만:===========================")
print("============================================================")
print(result.results["expert"].result)

### 병렬 처리

이제 문제의 두 가지 다른 측면을 보는 2개의 에이전트가 요청을 처리하고, 제공된 입력을 기반으로 요약 및 위험 계산을 담당하는 최종 에이전트에게 입력하는 토폴로지를 만들어보겠습니다
<div style="text-align:left">
    <img src="images/parallel.png" width="55%" />
</div>

In [ ]:
# agent_graph 기능을 가진 에이전트 초기화
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# 전문화된 에이전트 생성

financial_advisor = Agent(name="financial_advisor", system_prompt="당신은 비용-편익 분석, 예산 영향, ROI 계산에 중점을 둔 재무 고문입니다. 다른 전문가들과 협력하여 포괄적인 재무 관점을 구축하세요.")
technical_architect = Agent(name="technical_architect", system_prompt="당신은 실현 가능성, 구현 과제, 기술적 위험을 평가하는 기술 아키텍트입니다. 다른 전문가들과 협력하여 기술적 실현 가능성을 보장하세요.")
market_researcher = Agent(name="market_researcher", system_prompt="당신은 시장 상황, 사용자 요구, 경쟁 환경을 분석하는 시장 연구원입니다. 다른 전문가들과 협력하여 시장 기회를 검증하세요.")
risk_analyst = Agent(name="risk_analyst", system_prompt="당신은 잠재적 위험, 완화 전략, 규정 준수 문제를 식별하는 위험 분석가입니다. 다른 전문가들과 협력하여 포괄적인 위험 평가를 보장하세요.")


# 그래프 구축
builder = GraphBuilder()

# 노드 추가
builder.add_node(financial_advisor, "finance_expert")
builder.add_node(technical_architect, "tech_expert")
builder.add_node(market_researcher, "market_expert")
builder.add_node(risk_analyst, "risk_analyst")

# 엣지(종속성) 추가
builder.add_edge("finance_expert", "tech_expert")
builder.add_edge("finance_expert", "market_expert")
builder.add_edge("tech_expert", "risk_analyst")
builder.add_edge("market_expert", "risk_analyst")


# 진입점 설정 (선택사항 - 지정하지 않으면 자동 감지됨)
builder.set_entry_point("finance_expert")

# 그래프 구축
graph = builder.build()

print("============================================================")
print("============================================================")

# 새로 구축된 그래프에서 작업 실행
result = graph("우리 회사는 새로운 AI 기반 고객 서비스 플랫폼 출시를 고려하고 있습니다. 초기 투자는 200만 달러이고 예상 3년 ROI는 150%입니다. 재무 평가는 어떻습니까?")
print("\n")
print("============================================================")
print("============================================================")

print(f"응답: {result}")

print("=============노드 실행 순서:=================================")
print("============================================================")

# 어떤 노드가 어떤 순서로 실행되었는지 확인
for node in result.execution_order:
    print(f"실행됨: {node.node_id}")

print("=============그래프 메트릭:=================================")
print("============================================================")


# 성능 메트릭 가져오기
print(f"총 노드 수: {result.total_nodes}")
print(f"완료된 노드 수: {result.completed_nodes}")
print(f"실패한 노드 수: {result.failed_nodes}")
print(f"실행 시간: {result.execution_time}ms")
print(f"토큰 사용량: {result.accumulated_usage}")


# 특정 노드의 결과 가져오기

print("재무 고문:")
print("============================================================")
print("============================================================")
print(result.results["finance_expert"].result)
print("\n")

print("기술 전문가:")
print("============================================================")
print("============================================================")
print(result.results["tech_expert"].result)
print("\n")

print("시장 연구원:")
print("============================================================")
print("============================================================")
print(result.results["market_expert"].result)
print("\n")

### 조건부 분기

요청을 분류하고 코드에서 정의하는 조건에 따라 기술 또는 비즈니스 에이전트로 요청을 라우팅하는 에이전트 그래프를 만들어보겠습니다.

두 가지 다른 프롬프트를 기반으로 이 그래프에서 노드 실행 순서와 실행된 노드 수의 차이점을 자세히 살펴보세요.

<div style="text-align:left">
    <img src="images/conditional.png" width="55%" />
</div>

In [ ]:
# agent_graph 기능을 가진 에이전트 초기화
from strands.multiagent import GraphBuilder

mesh_agent = Agent()
# 전문화된 에이전트 생성

classifier = Agent(name="classifier", system_prompt="당신은 보고서 요청의 분류를 담당하는 에이전트입니다. Technical 또는 Business 분류만 반환하세요.")
technical_report = Agent(name="technical_expert", system_prompt="당신은 기술적 관점에서 간단한 요약을 제공하는 데 중점을 둔 기술 전문가입니다")
business_report = Agent(name="business_expert", system_prompt="당신은 비즈니스 관점에서 간단한 요약을 제공하는 데 중점을 둔 비즈니스 전문가입니다")

# 그래프 구축
builder = GraphBuilder()

# 노드 추가
builder.add_node(classifier, "classifier")
builder.add_node(technical_report, "technical_report")
builder.add_node(business_report, "business_report")

def is_technical(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "technical" in result_text.lower()

def is_business(state):
    classifier_result = state.results.get("classifier")
    if not classifier_result:
        return False
    result_text = str(classifier_result.result)
    return "business" in result_text.lower()

# 엣지(종속성) 추가
builder.add_edge("classifier", "technical_report", condition=is_technical)
builder.add_edge("classifier", "business_report", condition=is_business)

# 진입점 설정 (선택사항 - 지정하지 않으면 자동 감지됨)
builder.set_entry_point("classifier")

# 그래프 구축
graph = builder.build()

print("============================================================")
print("============================================================")

# 새로 구축된 그래프에서 작업 실행
result = graph("재택근무의 기술적 측면에 대한 보고서를 제공하고, 고려해야 할 사항과 주요 위험 요소를 개략적으로 설명하세요")
print("\n")
print("============================================================")
print("============================================================")

print(f"응답: {result}")

print("=============노드 실행 순서:=================================")
print("============================================================")

# 어떤 노드가 어떤 순서로 실행되었는지 확인
for node in result.execution_order:
    print(f"실행됨: {node.node_id}")

print("=============그래프 메트릭:=================================")
print("============================================================")


# 성능 메트릭 가져오기
print(f"총 노드 수: {result.total_nodes}")
print(f"완료된 노드 수: {result.completed_nodes}")
print(f"실패한 노드 수: {result.failed_nodes}")
print(f"실행 시간: {result.execution_time}ms")
print(f"토큰 사용량: {result.accumulated_usage}")

# 특정 노드의 결과 가져오기

print("분류기:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

# 새로 구축된 그래프에서 작업 실행
result = graph("재택근무의 비즈니스 영향에 대한 보고서를 제공하고, 고려해야 할 사항과 주요 위험 요소를 개략적으로 설명하세요")
print("\n")
print("============================================================")
print("============================================================")

print(f"응답: {result}")

print("=============노드 실행 순서:=================================")
print("============================================================")

# 어떤 노드가 어떤 순서로 실행되었는지 확인
for node in result.execution_order:
    print(f"실행됨: {node.node_id}")

print("=============그래프 메트릭:=================================")
print("============================================================")


# 성능 메트릭 가져오기
print(f"총 노드 수: {result.total_nodes}")
print(f"완료된 노드 수: {result.completed_nodes}")
print(f"실패한 노드 수: {result.failed_nodes}")
print(f"실행 시간: {result.execution_time}ms")
print(f"토큰 사용량: {result.accumulated_usage}")

# 특정 노드의 결과 가져오기

print("분류기:")
print("============================================================")
print("============================================================")
print(result.results["classifier"].result)
print("\n")

## 주요 요점 및 모범 사례


### 모범 사례:

**비순환성을 위한 설계:** 그래프에 순환이 없도록 보장하세요</p>
**의미 있는 노드 ID 사용:** 노드에 대해 설명적인 이름을 선택하세요</p>
**그래프 구조 검증:** 빌더가 순환을 확인하고 진입점을 검증합니다</p>
**노드 실패 처리:** 한 노드의 실패가 전체 워크플로에 미치는 영향을 고려하세요</p>
**조건부 엣지 사용:** 중간 결과에 기반한 동적 워크플로를 위해</p>
**병렬성 고려:** 독립적인 분기는 동시에 실행될 수 있습니다</p>
**멀티 에이전트 패턴 중첩:** 복잡한 워크플로를 위해 그래프 내에서 스웜 사용</p>
**멀티모달 입력 활용:** 이미지를 포함한 풍부한 입력을 위해 ContentBlocks 사용</p>

## 결론

이제 Strands Agent Graph로 멀티 에이전트 시스템을 구축하는 기본 사항을 마스터했습니다! 복잡한 문제를 해결하기 위해 협업하는 전문화된 AI 에이전트의 정교한 네트워크를 만들 수 있습니다.

성공적인 멀티 에이전트 시스템의 핵심은:
- 사용 사례에 토폴로지 매칭
- 명확한 에이전트 역할과 책임 정의  
- 적절한 통신 패턴 설정
- 리소스 관리 및 정리 효과적으로 수행

여기서부터 연구, 콘텐츠 생성, 의사 결정, 고객 서비스 등 실제 애플리케이션을 위한 점점 더 정교한 시스템을 구축할 수 있습니다.